In [1]:
import time

from data.swarmset import SwarmDataset, DataBuilder

baseline_data = DataBuilder("data/new")
baseline_data.create()
baseline_data.evolution.close()

similar_data = DataBuilder("data/similar", is_similar=True)
similar_data.create()
similar_data.evolution.close()

anti_data = DataBuilder("data/anti", is_anti=True)
anti_data.create()
anti_data.evolution.close()

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
0.0% Complete
0.20833333333333334% Complete
0.4166666666666667% Complete
0.625% Complete
0.8333333333333334% Complete
1.0416666666666667% Complete
1.25% Complete
1.4583333333333333% Complete
1.6666666666666667% Complete
1.875% Complete
2.0833333333333335% Complete
2.2916666666666665% Complete
2.5% Complete
2.7083333333333335% Complete
2.9166666666666665% Complete
3.125% Complete
3.3333333333333335% Complete
3.5416666666666665% Complete
3.75% Complete
3.9583333333333335% Complete
4.166666666666667% Complete
4.375% Complete
4.583333333333333% Complete
4.791666666666667% Complete
5.0% Complete
5.208333333333333% Complete
5.416666666666667% Complete
5.625% Complete
5.833333333333333% Complete
6.041666666666667% Complete
6.25% Complete
6.458333333333333% Complete
6.666666666666667% Complete
6.875% Complete
7.083333333333333% Complete
7.291666666666667% Complete
7.5% Complete
7.70

In [2]:
from data.swarmset import SwarmDataset, DataBuilder
from networks.encoder import BehaviorAutoEncoder
import numpy as np
import cv2
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = SwarmDataset("data/new")

encoder = BehaviorAutoEncoder()
encoder.load_model("cp_C-150")
encoder.to(device)
encoder.eval()

def resizeInput(X):
    frame = X.astype(np.uint8)
    resized = cv2.resize(frame, dsize=(200, 200), interpolation=cv2.INTER_CUBIC)
    return resized

def imgToEncoded(X):
    X = torch.Tensor(resizeInput(X)).to(device)
    X = torch.reshape(X, (1, 200*200))
    f_X = encoder.encoded(X).to(device)
    return X, f_X

def imgToDecoded(X):
    X = torch.Tensor(resizeInput(X)).to(device)
    X = torch.reshape(X, (1, 200*200))
    d = encoder(X).to(device)
    d = torch.reshape(d, (200, 200))
    return X, d

def encode_dataset(dataset):
    print("Encoding Dataset!")
    for i, (image, genome, behavior, _, _, _, _) in enumerate(dataset):
        _, encoded_val = imgToEncoded(image)
        encoded_val = encoded_val.squeeze(0).cpu().detach().numpy()
        _, decoded = imgToDecoded(image)
        decoded = decoded.cpu().detach().numpy()
        dataset.add_rank(i, encoded_val, is_array=True)
        dataset.add_image(i, decoded)
    dataset.set_rank(1)

baseline_data = SwarmDataset("data/new")
anti_data = SwarmDataset("data/anti")
similar_data = SwarmDataset("data/similar")

encode_dataset(baseline_data)
encode_dataset(anti_data)
encode_dataset(similar_data)

print("Done!")

Encoding Dataset!
Encoding Dataset!
Encoding Dataset!
Done!


# Embed into Latent Space

In [ ]:
import torch
from data.swarmset import SwarmDataset, DataBuilder
from networks.embedding import NoveltyEmbedding
from generation.halted_evolution import HaltedEvolution
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive
from NovelSwarmBehavior.novel_swarms.config.ResultsConfig import ResultsConfig
from NovelSwarmBehavior.novel_swarms.results.results import main as results
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults

TRAIN = True
CLUSTER_AND_DISPLAY = True
WRITE_OUT = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = NoveltyEmbedding().to(device)
anchor_dataset = SwarmDataset("data/new", rank=1)
anti_dataset = SwarmDataset("data/anti", rank=1)
similar_dataset = SwarmDataset("data/similar", rank=1)
evolution, screen = HaltedEvolution.defaultEvolver(steps=200, n_agents=10)

def shakeGenome(genome):
    return genome + (np.random.rand(len(genome)) * 0.01)

def antiGenome(genome):
    return -1 * shakeGenome(genome)

optimizer = torch.optim.Adam(network.parameters(), lr=1e-2)
loss_fn = torch.nn.TripletMarginLoss(margin=100)

if TRAIN:
    EPOCHS = 12
    for epoch in range(EPOCHS):

        easy_count = 0
        semi_hard_count = 0
        hard_count = 0
        none_count = 0

        for i, (_, _, _, anchor_encoding, _, _, _) in enumerate(anchor_dataset):
            _, _, _, similar_encoding, _, _, _ = similar_dataset[i]
            _, _, _, anti_encoding, _, _, _ = anti_dataset[i]

            anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
            similar_encoding = torch.from_numpy(similar_encoding).to(device).float()
            anti_encoding = torch.from_numpy(anti_encoding).to(device).float()

            optimizer.zero_grad()
            anchor_out = network(anchor_encoding)
            pos_out = network(similar_encoding)
            neg_out = network(anti_encoding)

            loss = loss_fn(anchor_out, pos_out, neg_out)
            loss.backward()
            optimizer.step()

            # if i % 10 == 0:
            #     print(f"{(i*100) / len(anchor_dataset)}%")

            with torch.no_grad():
                d_a_n = torch.linalg.norm(neg_out - anchor_out)
                d_a_p = torch.linalg.norm(pos_out - anchor_out)
                if d_a_n >= (d_a_p + loss_fn.margin):
                    easy_count += 1
                elif d_a_n <= d_a_p:
                    hard_count += 1
                elif d_a_p < d_a_n < (d_a_p + loss_fn.margin):
                    semi_hard_count += 1
                else:
                    none_count += 1


        print(f"Epoch: {epoch}")
        print(f"None Count: {none_count}")
        print(f"Easy Count: {easy_count}")
        print(f"Semi-Hard Count: {semi_hard_count}")
        print(f"Hard Count: {hard_count}")

if CLUSTER_AND_DISPLAY:
    network.eval()
    archive = NoveltyArchive()
    for i, (_, genome, _, anchor_encoding, _, _, _) in enumerate(anchor_dataset):
        anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
        embedding = network(anchor_encoding).squeeze(0).cpu().detach().numpy()
        archive.addToArchive(vec=embedding, genome=genome)

    print(len(archive.archive))
    print(len(archive.genotypes))
    print(archive.archive)

    agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
    world_config = ConfigurationDefaults.RECTANGULAR_WORLD
    world_config.addAgentConfig(agent_config)
    config = ResultsConfig(archive=archive, k_clusters=5, world_config=world_config, tsne_perplexity=2, tsne_early_exaggeration=8.0)
    results(config)

if WRITE_OUT:
    i = input("QUIT NOW IF YOU DONT WANT TO WRITE OUT")
    print("Writing in 3 seconds...")
    time.sleep(2)
    print("Writing in 1 seconds...")
    time.sleep(1)
    print("Writing out.")
    network.eval()
    for i, (_, _, _, anchor_encoding, _, _, _) in enumerate(anchor_dataset):
        anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
        embedding = network(anchor_encoding).squeeze(0).cpu().detach().numpy()
        anchor_dataset.add_rank(i, embedding, is_array=True)
    anchor_dataset.set_rank(2)

evolution.close()

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
Epoch: 0
None Count: 0
Easy Count: 21
Semi-Hard Count: 10
Hard Count: 449
Epoch: 1
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
Epoch: 2
None Count: 0
Easy Count: 62
Semi-Hard Count: 3
Hard Count: 415
Epoch: 3
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
Epoch: 4
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
Epoch: 5
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
Epoch: 6
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
Epoch: 7
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
Epoch: 8
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
Epoch: 9
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
Epoch: 10
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
Epoch: 11
None Count: 0
Easy Count: 0
Semi-Hard Count: 0
Hard Count: 480
480
480
[[-0.01607

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


TSNE Finished!
Starting k-Medoids Clustering
k-Medoids Finished!


/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:275: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(
/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:275: UserWarning: Cluster 2 is empty! self.labels_[self.medoid_indices_[2]] may not be labeled with its corresponding cluster (2).
  warnings.warn(
/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:275: UserWarning: Cluster 3 is empty! self.labels_[self.medoid_indices_[3]] may not be labeled with its corresponding cluster (3).
  warnings.warn(
/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:275: UserWarning: Cluster 4 is empty! self.labels_[self.medoid_indices_[4]] 